In [1]:
import pandas as pd

In [2]:
baseline = pd.read_json('./tests/fixture/baseline.json')
target = pd.read_json('./tests/fixture/target.json')
shield = pd.read_json('./tests/fixture/shield.json')
remap = pd.read_json('./tests/fixture/remap.json')

In [3]:
target['help_col'] = target['file_name']

In [4]:
merged = baseline.merge(target, how='left', on='file_name')

In [5]:
merged['status'] = 'passed'

In [6]:
merged.loc[merged['help_col'].isnull(), 'status'] = 'missed'

In [7]:
shield.rename(columns={'remark': 'shield_remark'}, inplace=True)
shield_file = shield.loc[shield['shield_type']=='file']
shield_dir = shield.loc[shield['shield_type']=='dir']

In [8]:
merged = merged.merge(shield_file, how='left', left_on='file_name', right_on='shield_path')

In [9]:
merged.loc[merged['shield_path'].notnull(),'status'] = 'shielded'

In [10]:
merged

,file_name,help_col,status,shield_path,shield_type,shield_remark
0,sit/per/thing/relationship/because.c,sit/per/thing/relationship/because.c,passed,NaN,NaN,NaN
1,big/operation/meeting/wear/firm.c,NaN,missed,NaN,NaN,NaN
2,discuss/quality/learn/green/option.c,NaN,missed,NaN,NaN,NaN
3,hour/rich/local/result/indicate.c,NaN,missed,NaN,NaN,NaN
4,raise/top/so/east/left.c,raise/top/so/east/left.c,passed,NaN,NaN,NaN
5,government/state/authority/box/in.c,NaN,missed,NaN,NaN,NaN
6,will/pull/cut/daughter/book.c,will/pull/cut/daughter/book.c,passed,NaN,NaN,NaN
7,develop/television/process/per/everyone.c,develop/television/process/per/everyone.c,passed,NaN,NaN,NaN
8,line/sure/your/type/one.c,NaN,missed,NaN,NaN,NaN
9,attorney/suggest/fall/notice/second.c,NaN,shielded,attorney/suggest/fall/notice/second.c,file,Not help today forget. Could yeah model outside.


In [11]:
shield_dir

,shield_path,shield_type,shield_remark
5,test_sheild_dir/test/a_dir,dir,Number fire read help.
6,test_sheild_dir/test_b,dir,When hospital civil growth.


In [37]:
missed = merged.loc[merged['status']=='missed',:].copy()

In [21]:
missed

,file_name,help_col,status,shield_path,shield_type,shield_remark
1,big/operation/meeting/wear/firm.c,NaN,missed,NaN,NaN,NaN
2,discuss/quality/learn/green/option.c,NaN,missed,NaN,NaN,NaN
3,hour/rich/local/result/indicate.c,NaN,missed,NaN,NaN,NaN
5,government/state/authority/box/in.c,NaN,missed,NaN,NaN,NaN
8,line/sure/your/type/one.c,NaN,missed,NaN,NaN,NaN
11,father/as/we/side/task.c,NaN,missed,NaN,NaN,NaN
13,attack/success/involve/arrive/environmental.c,NaN,missed,NaN,NaN,NaN
20,language/hard/strong/type/east.c,NaN,missed,NaN,NaN,NaN
25,yes/realize/institution/each/player.c,NaN,missed,NaN,NaN,NaN
28,fine/day/maybe/piece/thus.c,NaN,missed,NaN,NaN,NaN


In [67]:
def shield_dir_check(row, shield_dir):
    for _, shield_item in shield_dir.iterrows():
        if row['file_name'].startswith(shield_item['shield_path']):
            row['status'] = 'shielded'
            row['shield_path'] = shield_item['shield_path']
            row['shield_type'] = shield_item['shield_type']
            row['shield_remark'] = shield_item['shield_remark']


In [68]:
missed.apply(shield_dir_check, args=(shield_dir,), axis=1)

1     None
2     None
3     None
5     None
8     None
11    None
13    None
20    None
25    None
28    None
30    None
31    None
32    None
33    None
dtype: object

In [70]:
shield_dir_hit = missed.loc[missed['status'] == 'shielded', :]

In [71]:
shield_dir_hit

,file_name,help_col,status,shield_path,shield_type,shield_remark
30,test_sheild_dir/test/a_dir/executive/billion/b...,NaN,shielded,test_sheild_dir/test/a_dir,dir,Number fire read help.
31,test_sheild_dir/test_b/perform/unit/stuff/seek...,NaN,shielded,test_sheild_dir/test_b,dir,When hospital civil growth.
32,test_sheild_dir/test/a_dir/office/size/range/w...,NaN,shielded,test_sheild_dir/test/a_dir,dir,Number fire read help.
33,test_sheild_dir/test_b/stuff/watch/number/arti...,NaN,shielded,test_sheild_dir/test_b,dir,When hospital civil growth.


In [75]:
merged.loc[merged['file_name'].isin(shield_dir_hit['file_name']), :] = shield_dir_hit

In [76]:
merged

,file_name,help_col,status,shield_path,shield_type,shield_remark
0,sit/per/thing/relationship/because.c,sit/per/thing/relationship/because.c,passed,NaN,NaN,NaN
1,big/operation/meeting/wear/firm.c,NaN,missed,NaN,NaN,NaN
2,discuss/quality/learn/green/option.c,NaN,missed,NaN,NaN,NaN
3,hour/rich/local/result/indicate.c,NaN,missed,NaN,NaN,NaN
4,raise/top/so/east/left.c,raise/top/so/east/left.c,passed,NaN,NaN,NaN
5,government/state/authority/box/in.c,NaN,missed,NaN,NaN,NaN
6,will/pull/cut/daughter/book.c,will/pull/cut/daughter/book.c,passed,NaN,NaN,NaN
7,develop/television/process/per/everyone.c,develop/television/process/per/everyone.c,passed,NaN,NaN,NaN
8,line/sure/your/type/one.c,NaN,missed,NaN,NaN,NaN
9,attorney/suggest/fall/notice/second.c,NaN,shielded,attorney/suggest/fall/notice/second.c,file,Not help today forget. Could yeah model outside.
